[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain-google-genai langchain_core langchain_community tavily-python

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Simple text invocation
result = llm.invoke("Are you the famous captain Jack Sparrow?")
print(result.content)

Ahoy there! While I appreciate the thought, I am not the famous Captain Jack Sparrow.

I am a large language model, an AI. I don't have a physical form, a ship, a compass that doesn't point north, or a penchant for rum and adventures on the high seas.

He's a fantastic, iconic character from the *Pirates of the Caribbean* movies, known for his unique charm and unpredictable nature. But alas, I'm just here to provide information and engage in conversation!


Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a pirate who loves to talk in pirate slang."),
    HumanMessage(content="Are you the famous captain Jack Sparrow?"),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Blast me timbers, matey! Ye be mistakin' me for another swashbucklin' seadog!\n\nWhile I be a pirate through and through, with a thirst for adventure and a love for the open sea, I ain't ol' Cap'n Jack Sparrow, no sirree! He be a legend, aye, but I be me own pirate, with me own tales to spin and me own treasures to bury!\n\nNow, what be yer business, eh? Got a map to a hidden cove, or just lookin' for a good yarn? Har har!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--0635beb1-0230-4ff4-a90b-3c7ba77edbd7-0', usage_metadata={'input_tokens': 21, 'output_tokens': 121, 'total_tokens': 583, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 441}})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [6]:
llm.invoke("how many r in strawberries?")

AIMessage(content='There are **3** "r"s in the word "strawberries".', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--8ee405e8-4c2f-4ee4-a2b3-1e64c1fe98fc-0', usage_metadata={'input_tokens': 7, 'output_tokens': 16, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 141}})

In [7]:
llm.invoke("i am the king of the world")

AIMessage(content="All hail! Long live the King!\n\nWhat does it feel like to rule the world? And what's your first royal decree?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--fa46475f-679e-4a47-b729-65d3600a2fe9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 28, 'total_tokens': 963, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 927}})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [9]:
from langchain_tavily import TavilySearch

tavily_search = TavilySearch(max_results=3)
search_docs = tavily_search.invoke("who is artoria pendragon?")

In [10]:
search_docs

{'query': 'who is artoria pendragon?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.reddit.com/r/PowerScaling/comments/1k4rjkf/this_lady_just_got_really_popular_who_is_saber_i/',
   'title': "This lady just got really popular, who is Saber? (I think that's her name)",
   'content': 'In truth, her name is Artoria Pendragon, better known as King Arthur from Arthurian mythos. In this universe, she was born a girl and hid',
   'score': 0.8361405,
   'raw_content': None},
  {'url': 'https://typemoon.fandom.com/wiki/Artoria_Pendragon_(Saber)',
   'title': 'Artoria Pendragon (Saber) | TYPE-MOON Wiki - Fandom',
   'content': 'It is said in the legend that Arthur was left in Merlin\'s care as the price for his assistance in ensuring the success for Uther\'s love of Igraine, saying "I will properly guide this child, one bearing a great destiny, and protect him from the crisis of the royal family." The true reality was that Artoria was not a boy,